# 8760 Notebook

### Imports

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import os
from typing import List, Union, Tuple
from datetime import timedelta
from climakitae.explore.vulnerability import cava_data
from climakitae.util.utils import add_dummy_time_to_wl
from climakitae.explore.threshold_tools import (
    get_block_maxima,
    get_return_value,
)
from phase2 import *
random.seed(42)

### Global Variables

In [ ]:
# Putting locations into a DataFrame
locations = {
    "goleta": (34.486652, -119.894310),
    "iv": (34.427741, -119.855950),
    "sb_mountains": (34.518453, -119.799645),
    "valencia": (34.432468, -118.571098),
    "castaic": (34.492141, -118.605053),
    "lebec": (34.680102, -118.789246),
}

df = pd.DataFrame.from_dict(locations, orient='index', columns=['lat', 'lon']).reset_index()
df.rename(columns={'index': 'location'}, inplace=True)

In [ ]:
# Santa Barbara area
sb = df.iloc[:3][['lat', 'lon']]
sb_locs = ['goleta', 'iv', 'sb_mountains']

# Santa Clarita area
sc = df.iloc[3:6][['lat', 'lon']]
sc_locs = ['valencia', 'castaic', 'lebec']

In [ ]:
# Set params for 1-in-X calculations
one_in_x = np.array([10, 100])
event_duration = (1, 'day')
distr = 'gev'

### Grabbing appropriate datasets

In [ ]:
### Retrieving saved files for SB/SC locations, or generating the data from scratch if not available locally
sb_files = [
    'one_in_10-100_temperature_raw_data_34495N_-119882W.nc',
    'one_in_10-100_temperature_raw_data_34434N_-11987W.nc',
    'one_in_10-100_temperature_raw_data_3452N_-119782W.nc'
]
sc_files = [
    'one_in_10-100_temperature_raw_data_34425N_-118559W.nc',
    'one_in_10-100_temperature_raw_data_34492N_-118616W.nc',
    'one_in_10-100_temperature_raw_data_34695N_-118788W.nc',
]

clean_raw_data_sb = retrieve_data(sb_locs, sb, sb_files, 'SB')
clean_raw_data_sc = retrieve_data(sc_locs, sc, sc_files, 'SC')

### Calculating 1-in-X events

In [ ]:
%%time
# Calculating the 1-in-X return values, and lower + upper CIs, for SB locations
all_one_in_x_sb = get_one_in_x(clean_raw_data_sb, one_in_x, event_duration, distr)
calc_data_sb = all_one_in_x_sb['return_value']

In [ ]:
%%time
# Calculating the 1-in-X return values, and lower + upper CIs, for SC locations
all_one_in_x_sc = get_one_in_x(clean_raw_data_sc, one_in_x, event_duration, distr)
calc_data_sc = all_one_in_x_sc['return_value']

In [ ]:
# Plotting the 1-in-X values by simulation for Santa Barbara locations
plot_retvals(calc_data_sb)

In [ ]:
# Plotting the 1-in-X values by simulation for Santa Clarita locations
plot_retvals(calc_data_sc)

In [ ]:
# Finding the median of the above plots along simulation dimension
plot_med_val_by_locs(calc_data_sb)

In [ ]:
plot_med_val_by_locs(calc_data_sc)

# Inserting `time` amount of 1-in-X timesteps into 8760s

### Setting a number of +/- days for `ramp up` and `ramp down`, creating modified 8760s, and plotting them 

In [ ]:
# Setting param for number of +/- number of days to 'ramp up' and 'ramp down' by
t = 7

#### Here, we will look into how you can create a `modified8760` using the calculated median times to insert these 1-in-X events, and using the times that a user passes in.

#### 1. Using calculated median times to insert 1-in-X events into

In [ ]:
# Create a modified 8760 that injects 1-in-X events into the median times these events occur by location
modified8760_sb = create_modified_8760(clean_raw_data_sb, all_one_in_x_sb, t)

In [ ]:
# Plot the modified 8760 to see the events inserted!
plot_modified8760s(modified8760_sb)

#### 2. Insert 1-in-X events at times that users pre-define

In [ ]:
# This creates a dummy DataArray that follows the data shape needed to specify custom times
dummy_times = create_empty_da(all_one_in_x_sb['return_value'], keep_dims=['location', 'one_in_x'])

In [ ]:
# Here, you can manually change the values from `custom_times` to set specific times for these `one_in_x` events to occur.
# `updates` can either be a list of dictionaries (if you're looking to set these different events at different times, or you can
# set `updates` to a singular number to have all the events be inserted at the same time

updates = [
    {'location': 'goleta', 'one_in_x': 10, 'value': 1000},
    {'location': 'iv', 'one_in_x': 100, 'value': 1000}
]

# OR YOU CAN DO
updates = 3000

custom_times = set_custom_times(dummy_times, updates)

In [ ]:
# Calculates the modified 8760 with 1-in-X events inserted at custom times
modified8760_sb = create_modified_8760(clean_raw_data_sb, all_one_in_x_sb, t, custom_times=custom_times)

In [ ]:
plot_modified8760s(modified8760_sb)

#### Do the same for SC, but at hour=6000

In [ ]:
# Create `dummy_times` DataArray for SC
dummy_times = create_empty_da(all_one_in_x_sc['return_value'], keep_dims=['location', 'one_in_x'])

In [ ]:
# Modify event timing here
updates = 6000
custom_times = set_custom_times(dummy_times, updates)

In [ ]:
# Create and plot modified 8760 for SC
modified8760_sc = create_modified_8760(clean_raw_data_sc, all_one_in_x_sc, t, custom_times=custom_times)
plot_modified8760s(modified8760_sc)